Used Cars Price Prediction using ML

Dataset used from kaggle which is scraped from car Dekho website

In [106]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
from scipy.stats import iqr 
from sklearn.preprocessing import LabelEncoder
from collections import Counter as c
from sklearn.metrics import r2_score,mean_absolute_error,confusion_matrix, classification_report  
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from sklearn.tree import DecisionTreeRegressor

In [107]:
df = pd.read_csv('D:\ML Stuff/car data.csv')
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [108]:
#checking null values
df.isnull().sum()


Car_Name         0
Year             0
Selling_Price    0
Present_Price    0
Kms_Driven       0
Fuel_Type        0
Seller_Type      0
Transmission     0
Owner            0
dtype: int64

In [109]:
df.shape

(301, 9)

In [110]:
df['Year'].unique()

array([2014, 2013, 2017, 2011, 2018, 2015, 2016, 2009, 2010, 2012, 2003,
       2008, 2006, 2005, 2004, 2007], dtype=int64)

In [111]:
df['Selling_Price'].unique()

array([ 3.35,  4.75,  7.25,  2.85,  4.6 ,  9.25,  6.75,  6.5 ,  8.75,
        7.45,  6.85,  7.5 ,  6.1 ,  2.25,  7.75,  3.25,  2.65,  4.9 ,
        4.4 ,  2.5 ,  2.9 ,  3.  ,  4.15,  6.  ,  1.95,  3.1 ,  2.35,
        4.95,  5.5 ,  2.95,  4.65,  0.35,  5.85,  2.55,  1.25,  1.05,
        5.8 , 14.9 , 23.  , 18.  , 16.  ,  2.75,  3.6 ,  4.5 ,  4.1 ,
       19.99,  6.95, 18.75, 23.5 , 33.  , 19.75,  4.35, 14.25,  3.95,
        1.5 ,  5.25, 14.5 , 14.73, 12.5 ,  3.49, 35.  ,  5.9 ,  3.45,
        3.8 , 11.25,  3.51,  4.  , 20.75, 17.  ,  7.05,  9.65,  1.75,
        1.7 ,  1.65,  1.45,  1.35,  1.2 ,  1.15,  1.11,  1.1 ,  1.  ,
        0.95,  0.9 ,  0.75,  0.8 ,  0.78,  0.72,  0.65,  0.6 ,  0.55,
        0.52,  0.51,  0.5 ,  0.48,  0.45,  0.42,  0.4 ,  0.38,  0.31,
        0.3 ,  0.27,  0.25,  0.2 ,  0.18,  0.17,  0.16,  0.15,  0.12,
        0.1 ,  5.75,  5.15,  7.9 ,  4.85, 11.75,  3.15,  6.45,  3.5 ,
        8.25,  5.11,  2.7 ,  6.15, 11.45,  3.9 ,  9.1 ,  4.8 ,  2.  ,
        5.35,  6.25,

In [112]:
df.describe()

,Year,Selling_Price,Present_Price,Kms_Driven,Owner
count,301.000000,301.000000,301.000000,301.000000,301.000000
mean,2013.627907,4.661296,7.628472,36947.205980,0.043189
std,2.891554,5.082812,8.644115,38886.883882,0.247915
min,2003.000000,0.100000,0.320000,500.000000,0.000000
25%,2012.000000,0.900000,1.200000,15000.000000,0.000000
50%,2014.000000,3.600000,6.400000,32000.000000,0.000000
75%,2016.000000,6.000000,9.900000,48767.000000,0.000000
max,2018.000000,35.000000,92.600000,500000.000000,3.000000


In [113]:
df.drop(['Seller_Type'],axis=1)

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Manual,0
...,...,...,...,...,...,...,...,...
296,city,2016,9.50,11.60,33988,Diesel,Manual,0
297,brio,2015,4.00,5.90,60000,Petrol,Manual,0
298,city,2009,3.35,11.00,87934,Petrol,Manual,0
299,city,2017,11.50,12.50,9000,Diesel,Manual,0


In [114]:
ohe = OneHotEncoder()
ohe.fit(df[['Car_Name','Fuel_Type','Transmission']])

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [115]:
column_trans = make_column_transformer((OneHotEncoder(categories=ohe.categories_),['Car_Name','Fuel_Type','Transmission']),remainder='passthrough')


In [116]:
LR = LinearRegression()

In [117]:
#pipe will first take in data transform the columns then apply lr
pipe = make_pipeline(column_trans,LR)

In [118]:
x = df.drop(['Selling_Price','Seller_Type'],axis=1)
y = df['Selling_Price']


In [119]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=0)
x_test

,Car_Name,Year,Present_Price,Kms_Driven,Fuel_Type,Transmission,Owner
223,verna,2015,9.400,61381,Diesel,Manual,0
150,TVS Apache RTR 180,2011,0.826,6000,Petrol,Manual,0
226,grand i10,2015,5.700,24678,Petrol,Manual,0
296,city,2016,11.600,33988,Diesel,Manual,0
52,innova,2017,19.770,15000,Diesel,Automatic,0
...,...,...,...,...,...,...,...
137,Hero Extreme,2013,0.787,16000,Petrol,Manual,0
227,i10,2011,4.430,57000,Petrol,Manual,0
26,swift,2013,5.870,55138,Petrol,Manual,0
106,Hyosung GT250R,2014,3.450,16500,Petrol,Manual,1


In [120]:
pipe.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['800', 'Activa 3g', 'Activa 4g', 'Bajaj  ct 100',
       'Bajaj Avenger 150', 'Bajaj Avenger 150 street',
       'Bajaj Avenger 220', 'Bajaj Avenger 220 dtsi',
       'Bajaj Avenger S...
       'vitara brezza', 'wagon r', 'xcent'], dtype=object),
                                                                            array(['CNG', 'Diesel', 'Petrol'], dtype=object),
                                                                            array(['Automatic', 'Manual'], dtype=object)],
                                                                drop=None,
                            

In [121]:
y_pred = pipe.predict(x_test)

In [122]:
r2_score(y_test,y_pred)

0.9545110399655288

In [123]:
le=LabelEncoder()

In [124]:
le=LabelEncoder()
tempp=df
#label encoding Car_Name Fuel_Type and Transmission
tempp['Car_Name']=le.fit_transform(tempp['Car_Name'])                 
tempp['Fuel_Type']=le.fit_transform(tempp['Fuel_Type'])
tempp['Transmission']=le.fit_transform(tempp['Transmission'])

In [125]:
#defining independent, dependent variables
x1=tempp[['Car_Name','Year','Present_Price','Kms_Driven','Fuel_Type','Transmission','Owner']].values
y1=tempp['Selling_Price'].values 

In [126]:
#splitting data
from sklearn.model_selection import train_test_split                                    
x1_train,x1_test,y1_train,y1_test=train_test_split(x1,y1,test_size=0.2,random_state=0)

In [127]:
# fit the regressor with x and y data
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(x1_train, y1_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [128]:
reg_pred = regressor.predict(x1_test)  

In [129]:
print('model accuracy:',regressor.score(x1_test, y1_test))
print('mean_absolute_error: ',mean_absolute_error(y1_test, reg_pred))
print("r2_score: ",r2_score(y1_test,reg_pred))

model accuracy: 0.9583558255657589
mean_absolute_error:  0.5937721311475409
r2_score:  0.9583558255657588


In [130]:
xgbrf = XGBRFRegressor(n_estimators=500, random_state=42)
xgbrf.fit(x1_train, y1_train)

XGBRFRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=0.8, colsample_bytree=1,
               enable_categorical=False, gamma=0, gpu_id=-1,
               importance_type=None, interaction_constraints='',
               learning_rate=1.0, max_delta_step=0, max_depth=6,
               min_child_weight=1, missing=nan, monotone_constraints='()',
               n_estimators=500, n_jobs=8, num_parallel_tree=500,
               objective='reg:squarederror', predictor='auto', random_state=42,
               reg_alpha=0, reg_lambda=1e-05, scale_pos_weight=1, subsample=0.8,
               tree_method='exact', validate_parameters=1, verbosity=None)

In [131]:
y_predXGBRF = xgbrf.predict(x1_test)

In [132]:
print('Mean Absolute Error: ', round(metrics.mean_absolute_error(y1_test, y_predXGBRF), 3))
print('Mean Squared Error: ', round(metrics.mean_squared_error(y1_test, y_predXGBRF), 3))
print('Root Mean Squared Error: ', round(np.sqrt(metrics.mean_squared_error(y1_test, y_predXGBRF)), 3))
print('R2 score: ', round(r2_score(y1_test, y_predXGBRF), 3))

Mean Absolute Error:  0.606
Mean Squared Error:  1.285
Root Mean Squared Error:  1.134
R2 score:  0.949


In [133]:
dtr=DecisionTreeRegressor(random_state=0,criterion='mae')
dtr.fit(x1_train,y1_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mae', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')

In [134]:
y_preddtr=dtr.predict(x1_test)
y_preddtr

array([ 5.95,  0.4 ,  4.4 , 10.25, 17.  ,  5.3 ,  3.1 ,  0.45,  3.45,
        4.5 ,  2.  ,  0.9 ,  4.4 ,  6.7 ,  7.75, 17.  ,  6.4 ,  4.1 ,
        0.45,  1.75,  2.95,  4.9 ,  4.5 , 11.25,  0.3 ,  0.75,  0.3 ,
        0.6 ,  0.45,  4.  ,  1.95,  6.  ,  0.45,  8.35,  3.35,  1.2 ,
        5.25,  6.  ,  0.15,  5.5 ,  9.25, 18.75,  4.9 ,  4.4 ,  5.5 ,
       11.25,  0.25,  0.75,  5.9 ,  6.4 ,  5.35,  3.1 ,  4.95, 23.  ,
        1.2 ,  1.35,  0.42,  3.  ,  3.9 ,  3.  ,  3.35])

In [135]:
r2_score(y1_test,y_preddtr)

0.9672239695191153